# Diffusion, tractography and structural connections in the human brain

We start by first obtaining the bids directory with all the derivatives. For the present purpose we need to pay attention to the gray matter structures in the subcortex. 

In [16]:
import os
import glob
import shutil

In [18]:
subjects = ["01", "02", "06" ,"07"]     # List of subjects in you dataset to process
sessions = [                            # Sessions per subject
    [""],
    [""],
    ["2"],
    ["","2"]
]

for s, sub in enumerate(subjects):
    for ss, ses in enumerate(sessions[s]):
        print(f"Processing sub-CON{sub} in ses-control{ses}:")
        print("---------------------------------------------")
        
        # Relevant directories per subject and session
        bids_original = f"./BIDS-dir/sub-CON{sub}/ses-control{ses}/anat/"
        bids_derivatives = f"./BIDS-dir/derivatives/sub-CON{sub}/ses-control{ses}/anat/"     
        if not os.path.exists(bids_derivatives):
            os.makedirs(bids_derivatives)
        
        # Original T1 file
        file = os.path.join(bids_original, f"sub-CON{sub}_ses-control{ses}_T1w.nii.gz")
        
        # Brain Extraction Tool
        masked_brain = os.path.join(bids_derivatives, f"sub-CON{sub}_ses-control{ses}_T1w_brain.nii.gz")
        os.system(f"bet {file} {masked_brain} -f .5 -g -.6")
        os.system(f"bet {masked_brain} {masked_brain} -f .35 -m")
        print("Brain extracted succesfully")

        # Registration to template
        reference = "../Utils/MNI152_T1_1mm_brain.nii.gz"
        registered = os.path.join(bids_derivatives, f"sub-CON{sub}_ses-control{ses}_T1w_brain_space-MNI152.nii.gz")
        transformation = os.path.join(bids_derivatives, f"sub-CON{sub}_ses-control{ses}_T1w_brain_matrix2MNI152.txt")
        os.system(f"flirt -in {masked_brain} -ref {reference} -omat {transformation}")
        os.system(f"flirt -in {masked_brain} -ref {reference} -out {registered} -applyxfm -init {transformation}")
        print("Registered to MNI152 succesfully")

        # Tissue segmentation
        os.system(f"fast -n 4 -I 25 -W 25 -H .25 {registered}")
        print("Segmented tissues succesfully")

        # We apply an extra layer of segmentation for subcortical gray matter structures
        # TODO: first fsl https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/FIRST/UserGuide
        subcorticalGM = os.path.join(bids_derivatives, f"sub-CON{sub}_ses-control{ses}_T1w_brain_space-MNI152_subcortical.nii.gz")
        os.system(f"run_first_all -i {registered} -o {subcorticalGM}")
        os.rename(
            f"{bids_derivatives}sub-CON{sub}_ses-control{ses}_T1w_brain_space-MNI152_subcortical_all_fast_firstseg.nii.gz",
            f"{bids_derivatives}sub-CON{sub}_ses-control{ses}_T1w_brain_space-MNI152_subcortex-GM.nii.gz"
        )
        # Let's clean the output files that we won't be needing for this
        to_delete = glob.glob(f"{bids_derivatives}*_subcortical*")
        for f in to_delete:
            try:
                os.remove(f)
            except:
                shutil.rmtree(f)
        to_delete = glob.glob(f"{bids_derivatives}*_to_std_sub*")
        for f in to_delete:
            os.remove(f)
        print("Subcortical structures segmented succesfully")
        print("+++++++++++++++++++++++++++++++++++++++++++++++")
        print("+++++++++++++++++++++++++++++++++++++++++++++++")

Processing sub-CON01 in ses-control:
---------------------------------------------
Brain extracted succesfully
Registered to MNI152 succesfully
Segmented tissues succesfully
Subcortical structures segmented succesfully
+++++++++++++++++++++++++++++++++++++++++++++++
+++++++++++++++++++++++++++++++++++++++++++++++
Processing sub-CON02 in ses-control:
---------------------------------------------
Brain extracted succesfully
Registered to MNI152 succesfully
Segmented tissues succesfully
Subcortical structures segmented succesfully
+++++++++++++++++++++++++++++++++++++++++++++++
+++++++++++++++++++++++++++++++++++++++++++++++
Processing sub-CON06 in ses-control2:
---------------------------------------------
Brain extracted succesfully
Registered to MNI152 succesfully
Segmented tissues succesfully
Subcortical structures segmented succesfully
+++++++++++++++++++++++++++++++++++++++++++++++
+++++++++++++++++++++++++++++++++++++++++++++++
Processing sub-CON07 in ses-control:
-----------------

Now comes the question of whether the mask will alow the streamlines to start or not. Where do they stop?

TODO: Think/read a proper way to seed and stop the fibers.